The project follows a systematic approach:

• Import and preprocess the dataset, including handling missing values, standardizing numerical features, and encoding categorical variables.\
• Conduct exploratory data analysis (EDA) to understand data distribution and visualize patterns relevant to flight cancellations.\
• Perform feature engineering and split the dataset into training, validation, and test sets (80-10-10).\
• Establish baseline models with logistic regression and decision tree models to provide an initial benchmark for performance.\
• Apply advanced model training using more complex models to improve prediction accuracy.\
• Conduct hyperparameter tuning to optimize model parameters for the best performance.\
• Evaluate the optimized models on the test set to measure real-world prediction accuracy.\
• Analyze feature importance to interpret model results, identifying keyfactors affecting flight cancellations.\
• Compiling findings, methodologies, and future work for the final report.\

## Step-1: Import and preprocess the dataset

- Import the dataset.
- Handle missing values.

#### Importing the dataset.

In [3]:
import pandas as pd
import os

# Check the current working directory
print(os.getcwd())
!ls
# Print the full path before '/app'
full_path = os.getcwd()
print(full_path.split('/app')[0])
# Import the flights.csv file into a pandas DataFrame
df_flights = pd.read_csv('flight_dataset/flights.csv')
print(df_flights.head())

/app
docker-compose.yml  flight_dataset	    README.md
Dockerfile	    project_notebook.ipynb  requirements.txt



/tmp/ipykernel_239/2681771179.py:11: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_flights = pd.read_csv('flight_dataset/flights.csv')


   YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  ...  ARRIVAL_TIME  \
0            ANC                 SEA                    5  ...         408.0   
1            LAX                 PBI                   10  ...         741.0   
2            SFO                 CLT                   20  ...         811.0   
3            LAX                 MIA                   20  ...         756.0   
4            SEA                 ANC                   25  ...         259.0   

   ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
0          -

#### exploring the dataset types and columns and null values

In [15]:
def explore_the_df(df):
    print(df.shape)
    print(df.info())

    null_counts = df.isnull().sum()
    # Calculate the percentage of null values
    null_percentage = [(x / len(df) * 100) for x in null_counts]

    # Combine the counts and percentages into a single DataFrame
    null_summary = pd.DataFrame({'Null Count': null_counts, 'Percentage': null_percentage})
    print(null_summary)

In [16]:
explore_the_df(df_flights)

(5819079, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED          

As the last 5 columns have more than 80% null values, we decide to drop those

In [18]:
# List of columns to drop
columns_to_drop = ['CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

# Drop the columns from the dataframe
df_flights2 = df_flights.drop(columns=columns_to_drop)

# Print the updated dataframe columns
explore_the_df(df_flights2)

(5819079, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED          

We found that if we drop 105071 rows, we get a dataset without any null or nan values. As 105071 is just 1.8% of the main dataset, so we decided to drop these rows. After dropping these, the new shape is (5714008, 25)

In [22]:
# Drop rows with any null values from df_flights2
df_flights3= df_flights2.dropna()

# Print the shape of the new dataframe to confirm the rows have been dropped
explore_the_df(df_flights3)

(5714008, 25)
<class 'pandas.core.frame.DataFrame'>
Index: 5714008 entries, 0 to 5819078
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             in

In [26]:
df_flights3.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0


## Step-2: make changes to columns, encode, standardize and add engineered columns.


saving the cleaned df in a csv file

In [ ]:
df_flights3.to_csv('flights_cleaned.csv', index=False)

#df_flights3.to_csv('C:/Users/fsani/OneDrive/Desktop/ML_projec_5232/flight_dataset/flights_cleaned.csv', index=False)